In [ ]:
# =====================
# 표준 라이브러리
# =====================
import sys
from pathlib import Path

# =====================
# 서드 파티 라이브러리
# =====================
import polars as pl

# =====================
# 경로 설정
# =====================
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / "data"

# Python 내장 src 모듈과의 충돌 방지
if str(PROJECT_ROOT) in sys.path:
    sys.path.remove(str(PROJECT_ROOT))
sys.path.insert(0, str(PROJECT_ROOT))


# =====================
# 로컬 모듈
# =====================
from src.loading import DataLoader
from src.utils.statistic import Chi2Test


In [ ]:
# 데이터 로딩
loader = DataLoader(
    output_file=DATA_DIR / 'silver' / 'maude_clustered.parquet'
)

# Polars 어댑터 설정
adapter = 'polars'
polars_kwargs = {
    'use_statistics': True,  # 통계 정보 활용
    'parallel': 'auto',      # 자동 병렬 처리
    'low_memory': False,     # 메모리 최적화 비활성화 (성능 우선)
    'rechunk': False,        # 데이터 재정렬 비활성화
    'cache': True,           # 중간 결과 캐싱 활성화
}

# LazyFrame으로 로드 (실제 연산은 collect() 호출 시 수행)
df = loader.load(adapter=adapter, **polars_kwargs)

# 고장 유형 탐색

## 데이터 필드 정보

### 결함 유형 (defect_type)
mdr_text 전처리를 통해 생성된 결함 유형 분류 컬럼입니다.

**결함 유형 카테고리**:
* FUNCTIONAL_FAILURE = "Functional Failure"
* MECHANICAL_STRUCTURAL = "Mechanical/Structural"
* ELECTRICAL_POWER = "Electrical/Power"
* SOFTWARE_INTERFACE = "Software/Interface"
* ALARM_ALERT = "Alarm/Alert"
* SENSOR_ACCURACY = "Sensor/Accuracy"
* COMMUNICATION_CONNECTIVITY = "Communication/Connectivity"
* LABELING_PACKAGING = "Labeling/Packaging"
* STERILITY_CONTAMINATION = "Sterility/Contamination"
* USER_HUMAN_FACTOR = "User/Human Factor"
* ENVIRONMENTAL_COMPATIBILITY = "Environmental/Compatibility"
* OTHER = "Other"
* UNKNOWN = "Unknown"

### 주요 분석 컬럼
- `defect_type`: 결함 유형 (13개 카테고리)
- `product_code`: 제품 코드
- `manufacturer_name`: 제조사 이름

**참고**: Unknown과 Other는 명확한 패턴 분석이 어려워 일부 분석에서 제외됩니다.

## 이상 사례 발생 top 10 제품코드 / 결함 유형 발생 비율

In [ ]:
# Top 10 제품 코드 추출
# - 이상 사례 발생 빈도가 높은 상위 10개 제품 코드를 선정
# - LazyFrame으로 유지하여 메모리 효율성 확보
top10_result = (
    df
    .group_by("product_code")
    .agg(pl.len().alias("total_count"))
    .sort("total_count", descending=True)
    .head(10)
)

# Top 10 제품 코드를 리스트로 변환 (필터링에 사용하기 위해 collect)
top10_product_codes = (
    top10_result
    .select(pl.col('product_code').unique())
    .collect()
    .to_series()
    .to_list()
)

In [ ]:
# 전체 데이터에서 결함 유형별 발생 비율 계산
# - 각 결함 유형이 전체에서 차지하는 비율을 퍼센트로 표시
ratio_result = (
    df
    .group_by("defect_type")  # 결함 유형별로 그룹화
    .agg(pl.len().alias("count"))
    .with_columns(
        (pl.col("count") / pl.col("count").sum() * 100)
        .round(2)
        .alias("percentage")
    )
    .sort("percentage", descending=True)
)

# 결과 출력 (collect하여 실제 연산 수행)
print(ratio_result.collect().to_pandas())

## 어떤 제품코드가 어떤 고장 유형이 많이 발생하는가?

In [ ]:
# 제품 코드별 주요 결함 유형 분석
# - Top 10 제품 코드에 대해 각 제품별 상위 3개 결함 유형과 비율 계산
product_code_result = (
    df
    .filter(pl.col("product_code").is_in(top10_product_codes))
    .group_by(["product_code", "defect_type"])
    .agg(pl.len().alias("count"))
    .sort("count", descending=True)
    .group_by("product_code")
    .head(3)  # 각 제품 코드별 상위 3개 결함 유형만 선택
    .with_columns(
        # 제품 코드별 전체 건수 대비 해당 결함 유형의 비율 계산
        (pl.col("count") / pl.col("count").sum().over("product_code") * 100)
        .round(2)
        .alias("percentage")
    )
    .sort(["product_code", "percentage"], descending=[False, True])
)

# 결과 출력 (일부만)
print(product_code_result.head().collect())

## 어느 제조사에서 어떤 고장 유형이 많이 발생하는가?

In [ ]:
# 제조사별 주요 결함 유형 분석
# - 각 제조사별로 상위 5개 결함 유형과 비율 계산
manufacturer_result = (
    df
    .filter(pl.col("manufacturer_name").is_not_null())  # NULL 제조사 제외
    .group_by(["manufacturer_name", "defect_type"])
    .agg(pl.len().alias("count"))
    .sort("count", descending=True)
    .group_by("manufacturer_name")
    .head(5)  # 각 제조사별 상위 5개 결함 유형만 선택
    .with_columns(
        # 제조사별 전체 건수 대비 해당 결함 유형의 비율 계산
        (pl.col("count") / pl.col("count").sum().over("manufacturer_name") * 100)
        .round(2)
        .alias("percentage")
    )
    .sort(["manufacturer_name", "percentage"], descending=[False, True])
)

# 결과 출력 (일부만)
manufacturer_result.head().collect()

In [ ]:
# 분석 가능한 제조사 필터링
# - Unknown/Other를 제외한 명확한 결함 유형만 포함
# - 최소 3건 이상의 사례가 있는 제조사만 선택 (통계적 유의성 확보)
manufacturer_result.filter(
    (pl.col("count") >= 3) & 
    (~pl.col("defect_type").is_in(["Unknown", "Other"]))
).unique("manufacturer_name")

# 통계 분석

## 통계 검정 개요

### 1) 카이제곱 검정 (Chi-Square Test)
- **목적**: 특정 제품코드에서 특정 결함이 과대표되는지 검증
- **귀무가설 (H₀)**: 결함 유형과 제품코드는 독립적이다
- **대립가설 (H₁)**: 결함 유형과 제품코드는 관련이 있다
- **적용 방법**:
    - 관측 빈도 vs 기대 빈도 비교
    - 예: 제품코드 A에서 '배터리 결함' 발생 빈도가 전체 평균 대비 유의하게 높은가?
- **결과 해석**: p < 0.05이면 특정 패턴이 존재함을 의미

### 2) 표준화 잔차 (Standardized Residuals)
- **목적**: 어떤 셀(제품-결함 조합)이 기대치를 크게 벗어나는지 정량화
- **공식**: `(관측값 - 기대값) / sqrt(기대값)`
- **해석 기준**: 
    - |잔차| > 2: 해당 조합이 통계적으로 유의하게 기대치와 다름
    - |잔차| > 3: 매우 강한 연관성 (이상치 수준)
    - 양수(+): 관측값이 기대값보다 많음 (과대 표현)
    - 음수(-): 관측값이 기대값보다 적음 (과소 표현)

### 3) 효과 크기 - Cramér's V
- **목적**: 통계적 유의성과 별개로 실질적 연관 강도 측정
- **범위**: 0 (완전 독립) ~ 1 (완전 연관)
- **해석 기준**:
    - V < 0.1: 매우 약한 관계
    - 0.1 ≤ V < 0.3: 약한 관계
    - 0.3 ≤ V < 0.5: 중간 관계
    - V ≥ 0.5: 강한 관계
- **장점**: p-value는 표본 크기에 영향받지만, Cramér's V는 실질적 중요성을 나타냄

In [ ]:
# 카이제곱 검정을 위한 데이터 준비
# - Top 10 제품 코드만 선택 (분석 대상 축소)
# - Unknown/Other 제외 (명확한 패턴 분석을 위해)
# - Pandas DataFrame으로 변환 (Chi2Test 클래스가 Pandas 기반)
df_filtered = (
    df
    .filter(
        (pl.col("product_code").is_in(top10_product_codes))
        & (~pl.col("defect_type").is_in(["Unknown", "Other"]))
    )
    .select(["product_code", "defect_type"])
    .collect()  # LazyFrame을 실제 DataFrame으로 변환
    .to_pandas()  # Pandas로 변환
)

In [ ]:
# 카이제곱 검정 수행
# - 제품 코드(product_code)와 결함 유형(defect_type) 간의 독립성 검정
# - 출력: 기대빈도, 표준화 잔차, Cramér's V, 사후분석 결과
chi2_tester = Chi2Test()
chi2_result = chi2_tester.execute(df_filtered, 'product_code', 'defect_type')

## 오즈비 (Odds Ratio)

### 개념
- **목적**: 특정 제품에서 특정 결함 발생 확률을 정량적으로 비교
- **공식**: `OR = (제품A에서 결함발생 오즈) / (다른 제품에서 결함발생 오즈)`
- **오즈(Odds)**: 사건 발생 확률 / 사건 미발생 확률

### 해석 방법
- **OR = 1**: 두 그룹 간 차이 없음
- **OR > 1**: 타겟 제품에서 해당 결함이 더 많이 발생
- **OR < 1**: 타겟 제품에서 해당 결함이 덜 발생

### 결과 예시
"제품 X의 배터리 결함 OR = 4.2 (95% CI: 3.1-5.7)"
- **해석**: 다른 제품 대비 4.2배 높은 확률로 배터리 결함 발생
- **95% CI**: 신뢰구간이 1을 포함하지 않으면 통계적으로 유의함

### 추가 검증
- 카이제곱 검정을 통해 특정 제품 vs 전체 제품의 오즈비 차이가 통계적으로 유의한지 확인 가능

In [ ]:
import numpy as np
import pandas as pd

def calculate_odds_ratio(
    contingency_table: pd.DataFrame,
    target_product: str,
    target_defect: str,
    chi2_p_value: float = None,
    alpha: float = 0.05
) -> dict:
    """
    분할표에서 특정 제품-결함 조합의 오즈비 계산
    
    Parameters
    ----------
    contingency_table : pd.DataFrame
        pd.crosstab으로 생성한 분할표
        index: 제품 코드, columns: 결함 유형
    target_product : str
        분석 대상 제품 코드
    target_defect : str
        분석 대상 결함 유형
    chi2_p_value : float, optional
        Chi2Test()의 p-value (제공 시 사용)
    alpha : float
        유의수준 (default: 0.05)
    
    Returns
    -------
    dict
        - odds_ratio: 오즈비
        - ci_lower: 95% 신뢰구간 하한
        - ci_upper: 95% 신뢰구간 상한
        - p_value: 카이제곱 검정 p-value
        - interpretation: 해석
        - a, b, c, d: 2x2 분할표 셀 값
    
    Example
    -------
    >>> contingency_table = pd.crosstab(df['product_code'], df['defect_type'])
    >>> chi2_result = chi2_tester.execute(df, 'product_code', 'defect_type')
    >>> result = calculate_odds_ratio(contingency_table, 'INS', '배터리결함', chi2_result.p_value)
    >>> print(f"OR = {result['odds_ratio']:.2f} (95% CI: {result['ci_lower']:.2f}-{result['ci_upper']:.2f})")
    OR = 4.17 (95% CI: 2.52-6.89)
    
    Notes
    -----
    2x2 분할표 자동 생성 구조:
                    타겟 결함    기타 결함
    타겟 제품          a           b
    기타 제품          c           d
    
    연속성 수정(Haldane-Anscombe Correction):
    - 0이 포함된 셀이 있으면 모든 셀에 0.5를 더함
    - 로그 오즈비 계산 시 무한대 값 방지
    """
    
    # 2x2 분할표 생성
    # a: 타겟 제품 & 타겟 결함
    a = contingency_table.loc[target_product, target_defect] if target_product in contingency_table.index and target_defect in contingency_table.columns else 0
    
    # b: 타겟 제품 & 기타 결함
    b = contingency_table.loc[target_product, :].sum() - a if target_product in contingency_table.index else 0
    
    # c: 기타 제품 & 타겟 결함
    c = contingency_table.loc[:, target_defect].sum() - a if target_defect in contingency_table.columns else 0
    
    # d: 기타 제품 & 기타 결함
    total = contingency_table.values.sum()
    d = total - a - b - c
    
    # 2x2 분할표 출력
    print(f"\n[2x2 분할표: {target_product} - {target_defect}]")
    print(pd.DataFrame(
        [[a, b], [c, d]],
        index=[f'{target_product}', '기타 제품'],
        columns=[f'{target_defect}', '기타 결함']
    ))
    print()
    
    # Haldane-Anscombe 연속성 수정 (0 처리)
    # - 0이 있으면 로그 오즈비 계산 시 무한대 발생 방지
    # - 모든 셀에 0.5를 더하는 표준 방법 사용
    if b == 0 or c == 0 or a == 0 or d == 0:
        a, b, c, d = a + 0.5, b + 0.5, c + 0.5, d + 0.5
    
    # 오즈비 계산: OR = (a*d) / (b*c)
    odds_ratio = (a * d) / (b * c)
    
    # 로그 오즈비의 표준오차 (SE)
    se_log_or = np.sqrt(1/a + 1/b + 1/c + 1/d)
    
    # 95% 신뢰구간 계산 (로그 스케일에서 계산 후 역변환)
    z_critical = 1.96  # 95% CI
    log_or = np.log(odds_ratio)
    ci_lower = np.exp(log_or - z_critical * se_log_or)
    ci_upper = np.exp(log_or + z_critical * se_log_or)
    
    # p-value (Chi2Test 결과 사용)
    p_value = chi2_p_value
    
    # 결과 해석
    if p_value is not None and p_value < alpha:
        if odds_ratio > 1:
            interpretation = f"'{target_product}'에서 '{target_defect}'이(가) 다른 제품 대비 {odds_ratio:.2f}배 높음 (p={p_value:.4f}, 유의함)"
        else:
            interpretation = f"'{target_product}'에서 '{target_defect}'이(가) 다른 제품 대비 {1/odds_ratio:.2f}배 낮음 (p={p_value:.4f}, 유의함)"
    elif p_value is not None:
        interpretation = f"오즈비가 유의하지 않음 (p={p_value:.4f})"
    else:
        interpretation = "p-value 미제공"
    
    return {
        'odds_ratio': odds_ratio,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper,
        'p_value': p_value,
        'interpretation': interpretation,
        'a': int(a) if a == int(a) else a,
        'b': int(b) if b == int(b) else b,
        'c': int(c) if c == int(c) else c,
        'd': int(d) if d == int(d) else d
    }

In [ ]:
# 오즈비 계산 예시
# - 특정 제품-결함 조합에 대한 오즈비 분석

# 1. 분할표 생성 (교차표)
contingency_table = pd.crosstab(
    df_filtered['product_code'], 
    df_filtered['defect_type']
)

# 2. 카이제곱 검정은 이미 수행됨 (chi2_result 사용)

# 3. 오즈비 계산
# - 분석할 제품 코드와 결함 유형을 지정하여 계산
result = calculate_odds_ratio(
    contingency_table=contingency_table,
    target_product='LGW',               # 분석 대상 제품 코드 (예시)
    target_defect='Sensor/Accuracy',    # 분석 대상 결함 유형 (예시)
    chi2_p_value=chi2_result.p_value    # 카이제곱 검정 p-value
)

# 결과 출력
print(f"\nOR = {result['odds_ratio']:.2f} (95% CI: {result['ci_lower']:.2f}-{result['ci_upper']:.2f})")
print(f"해석: {result['interpretation']}")